Most examples work across multiple plotting backends, this example is also available for:

* [Bokeh NYC Airport Connection](../bokeh/nyc_airport_connections.ipynb)

In [ ]:
import networkx as nx
import numpy as np
from bokeh.sampledata.airport_routes import airports, routes

import holoviews as hv
from holoviews import dim, opts
from holoviews.element.graphs import layout_nodes

hv.extension('matplotlib')
hv.output(fig='svg', size=300)

## Declare data

In [ ]:
# Create dataset indexed by AirportID and with additional value dimension
airports = hv.Dataset(airports, ['AirportID'], ['Name', 'IATA', 'City'])
source_airports = list(airports.select(City='New York').data.AirportID)

# Add connections count to routes then aggregate and select just routes connecting to NYC
routes['connections'] = 1
nyc_graph = hv.Graph((routes, airports), ['SourceID', "DestinationID"], ['connections'], label='NYC Airport Connections')\
    .aggregate(function=np.count_nonzero).select(SourceID=source_airports)

# Lay out graph weighting and weight by the number of connections
np.random.seed(14)
graph = layout_nodes(nyc_graph, layout=nx.layout.fruchterman_reingold_layout, kwargs={'weight': 'connections'})
labels = hv.Labels(graph.nodes, ['x', 'y'], ['IATA', 'City'])

## Plot

In [ ]:
nyc_labels = labels.select(City='New York').opts(
    color='white', yoffset=0.05, size=16)

other_labels = labels[labels['City']!='New York'].opts(
    color='white', size=8)

cmap = {3697: 'red', 3797: 'blue'}

(graph * nyc_labels * other_labels).opts(
    opts.Graph(
        bgcolor='gray', xaxis=None, yaxis=None,
        edge_color=dim('SourceID').categorize(cmap, 'gray'),
        node_color=dim('index').categorize(cmap, 'gray'),
        title='NYC Airport Connections')
)